# ChIP AML PiPeline v2

In [ ]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../..')
import itertools
from scipy import stats
import numpy as np

from JKBio.epigenetics import ChIP_helper as chiphelper
from JKBio import Helper as helper
import igv
import SimpSOM as sps
from scipy import stats

import seaborn as sns
from matplotlib import cm
from matplotlib import pyplot as plt
from IPython.display import IFrame
import seaborn as sns
from bokeh.plotting import *
import igv

import numba
from numba import jit

from scipy.cluster.hierarchy import linkage, leaves_list
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from IPython.display import IFrame

from pybedtools import BedTool
import pyBigWig

output_notebook()
%load_ext autoreload
%autoreload 2

In [ ]:
project="Cobinding_ChIP"
version="v2"
merging_version="remove_single"
window="150"

In [ ]:
%store -r merged
%store -r promoters
#ATAC
%store -r a 
%store -r chrombed
%store -r mergedpeak
%store -r endata
%store -r l

## Correlation with Annotations

In [ ]:
data = pd.DataFrame(np.corrcoef(stats.zscore(0.01+merged[merged.columns[8:]].T)), columns=merged.columns[8:], index = merged.columns[8:])
link = linkage(data[:-endata])
col = data.iloc[-endata:-states]
col = col[[co for co in col.columns if co not in col.index.tolist()]]
rdb = cm.get_cmap('RdBu_r', 256)
for val in col.columns:
    a = [rdb(128+int(v*128)) for v in col[val]]
    col[val] = a

In [ ]:
len(data.columns), len(link)+len(range(24,46))

In [ ]:
#correlation_withannotation
fig = sns.clustermap(data.iloc[:-endata][data.columns[np.concatenate((leaves_list(link),range(24,38)))]], vmin=-1,vmax=1, col_colors=col.T, figsize=(20,20), cmap='RdBu_r', row_linkage=link, col_cluster=False, colors_ratio=0.015)
fig.ax_col_dendrogram.set_visible(False)
fig.fig.suptitle("correlation_withannotation")
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_correlation_with_annotation.pdf')
plt.show()

## Looking at peak overlap 

How many of peak in A (column) overlaps with peak in B (rows)

in other words:

what is the percentage of B's peaks that are overlaped by A's peaks 

In [ ]:
overlap, correlation, enrichment = chiphelper.pairwiseOverlap(merged, norm=True)

In [ ]:
overlap.to_csv('../results/'+project+'/'+version+'_'+merging_version+'_'+window+'pairwise_overlap.csv')
correlation.to_csv('../results/'+project+'/'+version+'_'+merging_version+'_'+window+'pairwise_correlation_on_overlap.csv')
enrichment.to_csv('../results/'+project+'/'+version+'_'+merging_version+'_'+window+'pairwise_enrichment_on_overlap.csv')

In [ ]:
data = pd.DataFrame(data=overlap,index=merged.columns[8:], columns=merged.columns[8:])
link = linkage(data.iloc[:-endata-states]) # D being the measurement
col = data[-endata-states+1:]
col = col[[co for co in col.columns if co not in col.index.tolist()]]
for val in col.columns:
    a = [viridis(v) for v in col[val]]
    col[val] = a

In [ ]:
len(data.columns),len(link), len(range(24,46))

In [ ]:
fig = sns.clustermap(data.iloc[:-endata-states+1][data.columns[np.concatenate((leaves_list(link),range(24,46)))]], row_linkage=link, col_colors=col.T, col_cluster=False,figsize=(20,20), colors_ratio=0.012)
fig.ax_col_dendrogram.set_visible(False)
fig.fig.suptitle("pairwise_overlap_clustermap")
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'pairwise_overlap_clustermap.pdf')
plt.show()

## data on the experiments

## Correlation only on overlaps 

on the overlaps given above

In [ ]:
data = pd.DataFrame(data=correlation,index=merged.columns[8:], columns=merged.columns[8:])
link = linkage(data.iloc[:-endata-states+1]) # D being the measurement
col = data.iloc[-endata-states+1:]
col = col[[co for co in col.columns if co not in col.index.tolist()]]
for val in col.columns:
    a = [rdb(128+int(v*128)) for v in col[val]]
    col[val] = a

In [ ]:
fig = sns.clustermap(data.iloc[:-endata-states+1][data.columns[np.concatenate((leaves_list(link),range(24,46)))]], row_linkage=link, col_colors=col.T, col_cluster=False, colors_ratio=0.012, figsize=(20,20), vmin=-1, vmax=1, cmap='RdBu_r')
fig.ax_col_dendrogram.set_visible(False)
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_correlation_onoverlap.pdf')
plt.show()

## Enrichment

In [ ]:
merged[merged.state_5==True]

In [ ]:
fig = sns.clustermap(enrichment,figsize=(12,12), vmin=-5,vmax=5, cmap='RdBu_r')
fig.ax_col_dendrogram.set_visible(False)
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_enrichment_clustermap_all_to_all.pdf')
plt.show()

## clustering

I have tried gaussian mixtures and Agglomerative clustering algorithm. Only the second can create a hierarchical clustering.

It seems that gaussian mixture makes more sense given the data we have, for now, is more "homogeneous". 

**I am still not so happy with the clustering.** It can be because of the how much importance, outlier values and the high number of noisy values from locations with no peaks.

We can use similar methods to RNAseq to improve this (clamping values, log transform, first round of PCA..)


### using DBSCAN

In [ ]:
cols = list(merged.columns[8:-endata-states+1])
data = merged[cols].values

In [ ]:
len(merged)

In [ ]:
rand = np.random.choice(range(len(data)),8000)

In [ ]:
data = ((data-data.min())/data.max(0))

In [ ]:
#print(merged.columns[8:-4])
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html#sklearn.cluster.OPTICS
min_samples=50
normtype="scaled"
groups = OPTICS(min_samples=min_samples,n_jobs=8).fit_predict(data[rand])
print(groups.max())
np.save('../data/'+project+'/'+version+'_'+merging_version+'_'+window+'_'+normtype+'_groups.npy',groups)

In [ ]:
groups = np.load('../data/'+project+'/'+version+'_'+merging_version+'_'+window+'_'+normtype+'_groups.npy')

In [ ]:
rand=5000
rand = np.random.choice(range(len(groups)),rand)

In [ ]:
subgroups = groups[rand]
print(subgroups.max())
sorting = np.argsort(subgroups)
print(len(set(subgroups)))
viridis = cm.get_cmap('gist_rainbow', len(set(groups)))
colors = [viridis(i) for i in subgroups[sorting]]

viridis = cm.get_cmap('viridis', 256)
data = merged[merged.columns[-11:]]
for val in data.columns[:-4]:
    data[val] =stats.zscore(np.log2(1+data[val]))
data = data.iloc[rand].iloc[sorting]
for val in data.columns:
    a = [viridis(v) for v in data[val]]
    data[val] = a
data["clusters"]  = colors

In [ ]:
fig = sns.clustermap(np.log2(1.01+merged[cols].iloc[rand].iloc[sorting].T), vmin=0,vmax=3,figsize=(20,15),z_score=0, colors_ratio=0.018, col_cluster=False, col_colors=data, xticklabels=False)
fig.ax_col_dendrogram.set_visible(False)
fig.fig.suptitle("sorted clustermap of cobindings clustered with OPTICS")
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'clustermap_cobinding_optics_minsamp_'+str(min_samples)+'_clustered.pdf')
plt.show()


### using KMeans

In [ ]:
cols = list(merged.columns[8:-endata])
data = stats.zscore(merged[cols].values)

In [ ]:
cols = list(merged.columns[8:-endata])
data= merged[cols].values
data = (data-data.min())/data.max()

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans
n_clust=20
kmean = KMeans(n_clusters=n_clust,n_jobs=8)
groups = kmean.fit_predict(data)
centroid = kmean.cluster_centers_

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
df = pd.DataFrame(centroid,columns=cols).T
df = df/df.max()

fig = sns.heatmap(df,ax=ax, vmax=1)
fig.figure.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_kmeans_'+str(n_clust)+'_centroids.pdf')

In [ ]:
merged.columns[-21:-7]

In [ ]:
subgroups = groups[rand]
sorting = np.argsort(subgroups)
viridis = cm.get_cmap('gist_rainbow', len(set(groups)))
colors = [viridis(i) for i in subgroups[sorting]]

viridis = cm.get_cmap('viridis', 256)
data = merged[merged.columns[-21:]]
for val in data.columns[:-8]:
    data[val] =stats.zscore(np.log2(1+data[val]))
data = data.iloc[rand].iloc[sorting]
for val in data.columns:
    a = [viridis(v*256) for v in data[val]]
    data[val] = a
data["clusters"]  = colors

In [ ]:
fig = sns.clustermap(np.log2(1.01+merged[cols].iloc[rand].iloc[sorting].T), vmin=0,vmax=3,figsize=(20,15),z_score=0, colors_ratio=0.018, col_cluster=False,col_colors=data, xticklabels=False)
fig.ax_col_dendrogram.set_visible(False)
fig.fig.suptitle("sorted clustermap of cobindings clustered with Kmeans")
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_kmeans_'+str(n_clust)+'_clustermap_cobinding.pdf')
plt.show()

In [ ]:
set(groups)

#### Enrichment

In [ ]:
enr, _ = chiphelper.enrichment(remerged,groups=groups)

In [ ]:
enr = enr.replace(-np.inf, -13.7)

In [ ]:
ig = sns.clustermap(enr.T[24:],figsize=(12,12), vmax=5, vmin=-5, cmap='RdBu_r')
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_kmeans_'+str(n_clust)+'enrichment_on_cluster_metasubset.pdf')
plt.show()

In [ ]:
fig = sns.clustermap(enr.T,figsize=(12,12), vmax=7, vmin=-7, cmap='RdBu_r')
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_kmeans_'+str(n_clust)+'enrichment_on_cluster.pdf')
plt.show()

## Plot TSNE density map

In [ ]:
cols = list(merged.columns[8:-21])
scaled_data = stats.zscore(merged[cols].values)
cols

In [ ]:
cols = list(merged.columns[8:-21])
data= merged[cols].values
scaled_data = (data - data.min(0))/data.max(0)

In [ ]:
rand = np.random.choice(merged.index,30000)

In [ ]:
red_data = TSNE(2,10,verbose=10,n_iter=1500).fit_transform(scaled_data)
np.save('../results/'+project+'/'+version+'_'+merging_version+'_'+window+'red_data.npy',red_data)

In [ ]:
# Let's look at the data. the density map of the distribution of all pseudo enhancer over their tf binding profile
_, ax = plt.subplots(figsize=(10,10))
fig = sns.kdeplot(red_data[:,0], red_data[:,1], shade=True, ax=ax)
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_density_TSNE.pdf')

In [ ]:
# now let's look at it from anothe rploting method. we can see many many islands
helper.bigScatter(red_data,binsize=0.2,showpoint=False,precomputed=False, logscale=True, title='density plot of enhancers in TF cobinding space with TSNE', folder='../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+"_")

In [ ]:
## TODO: color by dependencies/pathways/expression.. after ABC model,

## Using SOMs

In [ ]:
cols

In [ ]:
#Import the library
size = 20
#Build a network 20x20 with a weights format taken from the raw_data and activate Periodic Boundary Conditions. 
net = sps.somNet(size,size, data, PBC=True)

#Train the network for 10000 epochs and with initial learning rate of 0.01. 
net.train(0.01, 10000)

#Save the weights to file
net.save('../results/'+project+'/'+version+'_'+merging_version+'_'+window+'_cobinding_SOMweights_'+str(size))

In [ ]:
net = sps.somNet(0,0, data, loadFile='../results/'+project+'/'+version+'_'+merging_version+'_'+window+'_cobinding_SOMweights_'+str(size), PBC=True)

In [ ]:
pathplot = "../results/"+project+'/plots/'+version+'_'+merging_version+'_'+window+"_somNets/"
! mkdir $pathplot

In [ ]:
#Print a map of the network nodes and colour them according to the first feature (column number 0) of the dataset
#and then according to the distance between each node and its neighbours.
for col in range(data.shape[1]):
    net.nodes_graph(colnum=col, printout=True, show=True, path="../results/"+project+'/plots/'+version+'_'+merging_version+'_'+window+"_somNets/", colname=cols[col])

In [ ]:
diffs = net.diff_graph(show=False, returns=True)
plt.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_cobinding_SOM_'+str(size)+'.pdf')

In [ ]:
somnodes = {'r':[],'q':[],'c':diffs,'features':[]} 
for i, node in enumerate(net.nodeList):
    somnodes['q'].append(node.pos[0]+(i%size)*0.535+(i//size)*0.055)
    somnodes['r'].append(-node.pos[1]-(i%size)*0.2)
    somnodes['features'].append([cols[i] for i in np.argsort(node.weights) if abs(node.weights[i])>0.4])
somnodes=pd.DataFrame(somnodes)

In [ ]:
for i, v in somnodes.iterrows():
    tot=""
    for e, j in enumerate(v.features):
        if e%5==4:
            tot+='\n'
        tot += " "+str(j)
    somnodes.loc[i, 'features'] = tot

In [ ]:
#interactive SOM with features with highest importance to the nodes, displayed when hovering
helper.bigScatter(somnodes,precomputed=True, features=True,binsize=1, title='Cobinding SOM cluster of '+str(size), folder='../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window)

In [ ]:
#Cluster the datapoints according to the Quality Threshold algorithm.
clusts = net.cluster(data, type='KMeans')

In [ ]:
sorting  = []
for i in clusts:
    sorting.extend(i)
sorting = np.array(sorting)

In [ ]:
rand = np.random.choice(merged.index,8000)
rand = np.sort(rand)

In [ ]:
viridis = cm.get_cmap('gist_rainbow', len(clusts))
colors=[]
for i, clust in enumerate(clusts):
    colors.extend([viridis(i)]*len(clust))
colors = np.array(colors)
viridis = cm.get_cmap('viridis', 256)
data = merged[merged.columns[-21:]]
for val in data.columns[:-8]:
    data[val] =stats.zscore(np.log2(1+data[val]))
data = data.iloc[sorting[rand]]
for val in data.columns:
    a = [viridis(v*256) for v in data[val]]
    data[val] = a
data["clusters"]  = [tuple(i) for i in colors[rand]]

In [ ]:
#sorted clustermap of cobindings clustered with SOM
fig = sns.clustermap(np.log2(1.01+merged[cols].iloc[sorting[rand]].T), vmin=0,vmax=3,figsize=(20,15),z_score=0, colors_ratio=0.018, col_cluster=False,col_colors=data, xticklabels=False)
fig.ax_col_dendrogram.set_visible(False)
fig.fig.suptitle("sorted clustermap of cobindings clustered with SOM")
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_SOM_'+str(n_clust)+'_clustermap_cobinding.pdf')
plt.show()

In [ ]:
len(clusts)

In [ ]:
groups = []
for i, val in enumerate(clusts):
    groups.extend([i]*len(val))

In [ ]:
#let's look at the enrichment over SOM clusters
groups = []
for i, val in enumerate(clusts):
    groups.extend([i]*len(val))
enr, _ = chiphelper.enrichment(merged[merged.columns[:-states]],groups=np.array(groups))
enr = enr.replace(-np.inf, -13.7)
ig = sns.clustermap(enr.T[24:],figsize=(12,12), vmax=4, vmin=-4, cmap='RdBu_r')
fig.savefig('../results/'+project+'/plots/'+version+'_'+merging_version+'_'+window+'_kmeans_'+str(n_clust)+'enrichment_on_SOMcluster_metasubset.pdf')
plt.show()